# How to have chains

# 1. Create a simple chain

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import warnings
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import initialize_agent
from langchain.tools.retriever import create_retriever_tool
from dotenv import load_dotenv, find_dotenv
import os
from langchain.chains import SimpleSequentialChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

warnings.filterwarnings("ignore")
load_dotenv(find_dotenv(), override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)

In [ ]:
from langchain.agents.load_tools import get_all_tool_names, load_tools

print(get_all_tool_names())

# Chromadb

In [8]:
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

collection_name = "chatbot"
client = chromadb.HttpClient(host="localhost", port=8000)
client.heartbeat()
langchain_chroma = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_function,
)

# Loading tools

In [ ]:
tools = load_tools(["ddg-search", "llm-math"], llm=llm)

# find the name and description of each tool
print(tools[0].name, tools[0].description)
print(tools[1].name, tools[1].description, "\n\n")

## tool name and description

#('duckduckgo_search',
# 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output",
                                  return_messages=True)
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    memory=memory,
    return_intermediate_steps=True,  # Make sure you set it to True
    verbose=True
)
print(agent.agent.llm_chain.prompt.template)

# Create a chain with an agent
This agent will be used to answer a list of embeddings questions and predefined QA, there will be a tool that will be watching the conversation and will trigger to create an order.
* create a prompt for the agent convesation
* create a retriever tool for the QA
* create a retriever tool for the inventory
* create a tool to identify the order

In [17]:
from langchain import hub
from langchain.agents import AgentExecutor, create_json_chat_agent
from langchain.agents import load_tools
from langchain.tools import BaseTool


class QaRetriever(BaseTool):
    name = "qa_retriever"
    description = "Buscar en la base de datos de QA"

    def _run(self, query: str) -> str:
        result = langchain_chroma.similarity_search_with_score(query, k=2)
        return result[0][0].page_content


tools = [QaRetriever()]
prompt = hub.pull("hwchase17/react-chat-json")

agent = initialize_agent(llm=llm, tools=tools, verbose=True)

agent.run("cuales son los horarios de atencion?")





> Entering new AgentExecutor chain...
I should use the qa_retriever tool to find out what services are available.
Action: qa_retriever
Action Input: servicios disponibles

KeyboardInterrupt: 